#### <b>KoCLIP 모델 불러오기</b>

In [ ]:
import torch
from transformers import AutoModel, AutoProcessor

# KoCLIP 모델 로드
repo = "Bingsu/clip-vit-large-patch14-ko"
model = AutoModel.from_pretrained(repo)
processor = AutoProcessor.from_pretrained(repo)

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(f"KoCLIP 모델이 {device}에서 로드됨")

#### <b>이미지 다운로드 및 특징 추출 예시 코드</b>

In [ ]:
import requests
from PIL import Image
from io import BytesIO

def download_actor_image(actor_id):
    """배우 ID에 해당하는 이미지를 다운로드하여 PIL 이미지로 반환"""
    img_url = f"https://playbuddy.s3.ap-northeast-2.amazonaws.com/actors/{actor_id}/{actor_id}_1.jpg"

    try:
        response = requests.get(img_url, timeout=5)
        if response.status_code == 404:
            print(f"배우 {actor_id}: 이미지 없음 (404)")
            return None

        image = Image.open(BytesIO(response.content)).convert("RGB")
        return image

    except Exception as e:
        print(f"배우 {actor_id}: 이미지 다운로드 실패 - {e}")
        return None

In [ ]:
import torch

def extract_image_features(image):
    """KoCLIP을 이용해 이미지 특징 벡터(768차원)를 추출"""
    try:
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features.cpu().numpy().flatten()

    except Exception as e:
        print(f"이미지 특징 벡터 추출 실패 - {e}")
        return None

In [ ]:
actor_ids = range(1, 8001)  # 1번부터 8000번까지

* 단순히 <b>매 이미지</b>를 일일이 <b>KoCLIP</b>에 넣어 결과 반환
* 정상적으로 동작하는 코드이지만, 너무 많은 시간이 소요됨

In [ ]:
import numpy as np

actor_vectors = {}

for actor_id in actor_ids:
    image = download_actor_image(actor_id)
    if image is None:
        continue  # 이미지가 없으면 건너뜀

    features = extract_image_features(image)
    if features is not None:
        actor_vectors[actor_id] = features

    # 100명 단위로 진행 상황 저장
    if actor_id % 100 == 0:
        np.save("actor_image_features.npy", actor_vectors)
        print(f"{actor_id}번까지 저장 완료")

# 최종 저장
np.save("actor_image_features.npy", actor_vectors)
print("모든 배우 벡터 저장 완료")

#### <b>임의의 500명에 대하여 배치 단위 특징 추출</b>

In [ ]:
import numpy as np

# 1번부터 8000번 사이에서 무작위로 500명의 배우 ID 선택
np.random.seed(42)  # 재현 가능성을 위해 시드 설정
actor_ids = np.random.choice(range(1, 8001), size=500, replace=False)

print(f"선택된 배우 ID 개수: {len(actor_ids)}")
print("일부 배우 ID 예시:", actor_ids[:10])  # 처음 10개만 출력

In [ ]:
import requests
from PIL import Image
from io import BytesIO

def download_actor_images(actor_batch):
    """이미지를 다운로드하여 배우 이미지 리스트(배치)에 삽입"""
    images, valid_actor_ids = [], []

    for actor_id in actor_batch:
        img_url = f"https://playbuddy.s3.ap-northeast-2.amazonaws.com/actors/{actor_id}/{actor_id}_1.jpg"

        try:
            response = requests.get(img_url, timeout=5)
            if response.status_code == 404:
                continue  # 이미지 없음

            image = Image.open(BytesIO(response.content)).convert("RGB")
            images.append(image)
            valid_actor_ids.append(actor_id)

        except Exception as e:
            print(f"배우 {actor_id}: 이미지 다운로드 실패 - {e}")

    return images, valid_actor_ids

In [ ]:
def extract_batch_features(images):
    """배치 단위로 이미지 특징 벡터를 추출"""
    try:
        inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features.cpu().numpy()

    except Exception as e:
        print(f"배치 특징 벡터 추출 실패 - {e}")
        return None

In [ ]:
batch_size = 16
actor_vectors = {}

for i in range(0, len(actor_ids), batch_size):
    actor_batch = actor_ids[i : i + batch_size]
    images, valid_actor_ids = download_actor_images(actor_batch)

    if not images:
        continue  # 이미지가 하나도 없으면 건너뜀

    features = extract_batch_features(images)
    if features is not None:
        for j, actor_id in enumerate(valid_actor_ids):
            actor_vectors[actor_id] = features[j]

    print(f"{i + len(valid_actor_ids)}명 처리 완료")

# 최종 저장
np.save("actor_image_features_500.npy", actor_vectors)
print("500명 배우 벡터 저장 완료")

#### <b>배우 이미지 유사도 측정 결과 확인</b>

In [ ]:
from scipy.spatial.distance import cosine

def find_similar_actors(target_actor_id, top_k=3):
    """특정 배우와 유사한 최대 top_k명의 배우 찾기"""
    if target_actor_id not in actor_vectors:
        print(f"배우 {target_actor_id}의 벡터 정보가 없습니다.")
        return []

    target_vector = actor_vectors[target_actor_id]
    similarities = []

    for actor_id, vector in actor_vectors.items():
        if actor_id == target_actor_id:
            continue
        similarity = 1 - cosine(target_vector, vector)  # 코사인 유사도 (1에 가까울수록 유사)
        similarities.append((actor_id, similarity))

    # 유사도 기준으로 정렬 후 상위 top_k명 반환
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

In [ ]:
import matplotlib.pyplot as plt

def show_similar_actors(target_actor_id):
    """특정 배우와 유사한 배우 3명의 사진을 시각화"""
    similar_actors = find_similar_actors(target_actor_id)

    if not similar_actors:
        print("유사 배우를 찾을 수 없습니다.")
        return

    actor_list = [target_actor_id] + [actor[0] for actor in similar_actors]
    fig, axes = plt.subplots(1, len(actor_list), figsize=(15, 5))

    for i, actor_id in enumerate(actor_list):
        img_url = f"https://playbuddy.s3.ap-northeast-2.amazonaws.com/actors/{actor_id}/{actor_id}_1.jpg"
        response = requests.get(img_url)
        if response.status_code == 404:
            axes[i].text(0.5, 0.5, f"Actor {actor_id} 없음", ha="center", va="center")
            axes[i].axis("off")
            continue

        image = Image.open(BytesIO(response.content)).convert("RGB")
        axes[i].imshow(image)
        axes[i].set_title(f"Actor {actor_id}")
        axes[i].axis("off")

    plt.show()

# 예제 실행 (유사 배우 찾기)
show_similar_actors(target_actor_id=2216)

In [ ]:
show_similar_actors(target_actor_id=2583)

In [ ]:
show_similar_actors(target_actor_id=1663)

In [ ]:
show_similar_actors(target_actor_id=3028)

In [ ]:
show_similar_actors(target_actor_id=2681)

In [ ]:
show_similar_actors(target_actor_id=4055)

In [ ]:
show_similar_actors(target_actor_id=3762)

#### <b>CSV에 특징 벡터 저장 및 불러와 테스트하기</b>

In [ ]:
import csv

csv_filename = "actor_image_features_500.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)

    # CSV 헤더
    writer.writerow(["actor_id", "profile_url", "features"])

    # 데이터 저장
    for actor_id, vector in actor_vectors.items():
        profile_url = f"https://playbuddy.s3.ap-northeast-2.amazonaws.com/actors/{actor_id}/{actor_id}_1.jpg"
        features_str = ",".join(map(str, vector))  # 리스트를 문자열로 변환
        writer.writerow([actor_id, profile_url, features_str])

print(f"CSV 파일 저장 완료: {csv_filename}")

In [ ]:
actor_vectors = {}

with open(csv_filename, mode="r") as file:
    reader = csv.reader(file)
    next(reader)  # 헤더 스킵

    for row in reader:
        actor_id = int(row[0])
        profile_url = row[1]
        features = np.array(list(map(float, row[2].split(","))))  # 문자열을 벡터로 변환
        actor_vectors[actor_id] = (profile_url, features)

print(f"CSV에서 {len(actor_vectors)}명의 배우 벡터 로드 완료")

In [ ]:
from scipy.spatial.distance import cosine

def find_similar_actors(target_actor_id, top_k=3):
    """특정 배우와 유사한 최대 top_k명의 배우 찾기"""
    if target_actor_id not in actor_vectors:
        print(f"배우 {target_actor_id}의 벡터 정보가 없습니다.")
        return []

    target_vector = actor_vectors[target_actor_id][1]  # 벡터만 가져오기
    similarities = []

    for actor_id, (profile_url, vector) in actor_vectors.items():
        if actor_id == target_actor_id:
            continue
        similarity = 1 - cosine(target_vector, vector)  # 코사인 유사도 (1에 가까울수록 유사)
        similarities.append((actor_id, profile_url, similarity))

    # 유사도 기준으로 정렬 후 상위 top_k명 반환
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_k]

In [ ]:
import matplotlib.pyplot as plt

def show_similar_actors(target_actor_id):
    """특정 배우와 유사한 배우 3명의 사진을 시각화"""
    similar_actors = find_similar_actors(target_actor_id)

    if not similar_actors:
        print("유사 배우를 찾을 수 없습니다.")
        return

    actor_list = [(target_actor_id, actor_vectors[target_actor_id][0])] + similar_actors
    fig, axes = plt.subplots(1, len(actor_list), figsize=(15, 5))

    for i, (actor_id, profile_url, *_) in enumerate(actor_list):
        response = requests.get(profile_url)
        if response.status_code == 404:
            axes[i].text(0.5, 0.5, f"Actor {actor_id} 없음", ha="center", va="center")
            axes[i].axis("off")
            continue

        image = Image.open(BytesIO(response.content)).convert("RGB")
        axes[i].imshow(image)
        axes[i].set_title(f"Actor {actor_id}")
        axes[i].axis("off")

    plt.show()

# 예제 실행
show_similar_actors(target_actor_id=3762)

#### <b>전체 이미지에 대한 특징 추출 예시</b>

In [ ]:
actor_ids = list(range(1, 8001))  # 1번부터 8000번까지
print(f"총 배우 수: {len(actor_ids)}")

In [ ]:
batch_size = 64
actor_vectors = {}

for i in range(0, len(actor_ids), batch_size):
    actor_batch = actor_ids[i : i + batch_size]
    images, valid_actor_ids = download_actor_images(actor_batch)

    if not images:
        continue  # 이미지가 하나도 없으면 건너뜀

    features = extract_batch_features(images)
    if features is not None:
        for j, actor_id in enumerate(valid_actor_ids):
            actor_vectors[actor_id] = features[j]

    print(f"{i + len(valid_actor_ids)}/{len(actor_ids)}명 처리 완료")

print("모든 배우 데이터 추출 완료!")

In [ ]:
csv_filename = "actor_image_features_8000.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["actor_id", "profile_url", "features"])

    for actor_id, vector in actor_vectors.items():
        profile_url = f"https://playbuddy.s3.ap-northeast-2.amazonaws.com/actors/{actor_id}/{actor_id}_1.jpg"
        features_str = ",".join(map(str, vector))
        writer.writerow([actor_id, profile_url, features_str])

print(f"CSV 파일 저장 완료: {csv_filename}")

#### <b>JavaScript를 활용하여 동일하게 이미지 유사도 분석</b>

* JavaScript 개발자에게 전달하기 위한 목적의 코드

In [ ]:
!apt-get install -y nodejs npm  # Node.js 및 npm 설치
!npm install csv-parser cosine-similarity  # 필요한 패키지 설치

In [ ]:
js_code = """
const fs = require("fs");
const csv = require("csv-parser");
const cosineSimilarity = require("cosine-similarity");

const actorData = {}; // { actor_id: { profileUrl, features } }

fs.createReadStream("actor_image_features_8000.csv")
  .pipe(csv())
  .on("data", (row) => {
    const actorId = parseInt(row.actor_id);
    const profileUrl = row.profile_url;
    const features = row.features.split(",").map(Number);
    actorData[actorId] = { profileUrl, features };
  })
  .on("end", () => {
    console.log(`총 ${Object.keys(actorData).length}명의 배우 데이터 로드 완료!`);
    findSimilarActors(1);  // 특정 배우와 유사한 배우 찾기 실행
  });

function findSimilarActors(targetActorId, topK = 3) {
  if (!actorData[targetActorId]) {
    console.log(`배우 ${targetActorId}ㅁ의 데이터가 없습니다.`);
    return;
  }

  const targetProfileUrl = actorData[targetActorId].profileUrl;
  console.log(`\\n🔹 배우 ${targetActorId}의 프로필 이미지: ${targetProfileUrl}\\n`);

  const targetVector = actorData[targetActorId].features;
  const similarities = [];

  for (const [actorId, { profileUrl, features }] of Object.entries(actorData)) {
    if (parseInt(actorId) === targetActorId) continue;
    const similarity = cosineSimilarity(targetVector, features);
    similarities.push({ actorId: parseInt(actorId), profileUrl, similarity });
  }

  similarities.sort((a, b) => b.similarity - a.similarity);
  const topActors = similarities.slice(0, topK);

  console.log(`배우 ${targetActorId}와 가장 유사한 ${topK}명의 배우:`);
  console.table(topActors);
}
"""

# JavaScript 코드 파일 생성
with open("actor_similarity.js", "w") as f:
    f.write(js_code)

print("✅ JavaScript 코드 파일 생성 완료: actor_similarity.js")

In [ ]:
%%shell
node actor_similarity.js